<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/38_3_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка необходимых пакетов
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libcudnn8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-tpu 2.18.0 requires keras>=3.5.0, which is not installed.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-tpu 2.18.0 requires keras>=3.5.0, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00


In [2]:
# Импорт необходимых библиотек
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text as text
import re
import pathlib
import random
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Загрузка датасета
examples, metadata = tfds.load('web_questions', with_info=True)
train_examples, test_examples = examples['train'], examples['test']

In [5]:
print('Размер обучающей выборки: ', len(train_examples))
print('Размер валидационной выборки: ', len(test_examples))

Размер обучающей выборки:  3778
Размер валидационной выборки:  2032


In [6]:
print(train_examples.dtype)

AttributeError: '_PrefetchDataset' object has no attribute 'dtype'

In [7]:
for q_examples, a_examples in train_examples.batch(3).take(1):
    print('Примеры:')
    print()
    for q in q_examples.numpy():
        print(q.decode('utf-8'))
    print()

    print('Примеры:')
    print()
    for en in a_examples.numpy():
        print(a.decode('utf-8'))

ValueError: too many values to unpack (expected 2)

In [8]:
# Токенизация списков вопросов и ответов
VOCAB_SIZE = 8000
bert_tokenizer_params=dict(lower_case=True)

reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    vocab_size = VOCAB_SIZE,
    reserved_tokens=reserved_tokens,
    bert_tokenizer_params=bert_tokenizer_params,
    learn_params={},)

In [9]:
train_question = train_examples.map(lambda x: x['question'])
train_answer = train_examples.map(lambda x: x['answers'][0])

print(train_question)
print(train_answer)

<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>
<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [10]:
question_vocab = bert_vocab.bert_vocab_from_dataset(train_question.batch(1000).prefetch(2), **bert_vocab_args)

print(question_vocab[:10])
print(question_vocab[100:110])
print(question_vocab[1000:1010])
print(question_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', "'", '(', ')', '.', '0', '1']
['use', 'college', '##ie', 'and', 'city', 'movies', '##k', '##h', '##m', 'state']
[]
['##3', '##4', '##5', '##6', '##7', '##8', '##9', '##?', '##j', '##q']


In [11]:
def write_vocab_file(filepath, vocab):
    with open(filepath, 'w') as f:
        for token in vocab:
            print(token, file=f)

write_vocab_file('question_vocab.txt', question_vocab)

In [12]:
answer_vocab = bert_vocab.bert_vocab_from_dataset(train_answer.batch(1000).prefetch(2),  **bert_vocab_args)

print(answer_vocab[:10])
print(answer_vocab[100:110])
print(answer_vocab[1000:1010])
print(answer_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '#', '&', "'", '(']
['york', '##ist', 'city', '##c', '##ch', '##ie', 'in', '##ion', 'canada', 'north']
[]
['##.', '##/', '##:', '##?', '##j', '##p', '##q', '##£', '##–', '##−']


In [13]:
write_vocab_file('answer_vocab.txt', answer_vocab)

In [14]:
!ls *.txt

answer_vocab.txt  question_vocab.txt


In [15]:
question_tokenizer = text.BertTokenizer('question_vocab.txt', **bert_tokenizer_params)
answer_tokenizer = text.BertTokenizer('answer_vocab.txt', **bert_tokenizer_params)

In [16]:
print('Отберем строки для примера:')
for pt_examples, en_examples in train_examples.batch(3).take(1):
  for ex in en_examples:
    print(ex.numpy())

Отберем строки для примера:


ValueError: too many values to unpack (expected 2)

In [17]:
### получается, что в уроке en_examples.dtype возвращает <dtype: 'string'>
print(train_answer.dtype)

AttributeError: '_MapDataset' object has no attribute 'dtype'

In [21]:
train_answer = tf.convert_to_tensor(train_answer)

# Токенизируем примеры и получаем тензор формы (batch, word, word-piece)
token_batch = answer_tokenizer.tokenize(train_answer)
# Объединяем оси word и word-piece и получаем тензор формы (batch, tokens)
token_batch = token_batch.merge_dims(-2,-1)

print('Токенизируем отобранные строки:')
for ex in token_batch.to_list():
  print(ex)

Токенизируем отобранные строки:
[37, 65, 63, 33, 206, 103, 368]
[50, 84, 67, 88, 134]
[48, 61, 103, 11, 16, 153, 26, 16, 111]
[29, 288, 210, 77]
[40, 212, 228, 50, 116, 59, 30, 116, 63]
[46, 65, 103, 205, 71, 65, 88, 64]
[31, 341, 266, 131, 49, 65, 103, 147, 87, 123, 89]
[237, 62, 80, 131, 129]
[18, 111, 111, 189, 40, 60, 438, 94, 413, 344, 126]
[33, 67, 438, 67, 35, 77, 76, 67]
[35, 60, 144, 265, 67]
[30, 131, 71, 43, 84, 186, 269, 439, 32, 172, 339, 29, 365]
[309, 7, 40, 63]
[43, 91, 62, 57, 64, 71, 233, 317, 124, 79]
[47, 212, 129, 219, 45, 63, 88, 107, 351, 41, 251, 46, 121, 67, 150, 76, 115, 29, 63, 59, 28, 76, 248, 62, 101, 210, 396, 166, 31, 67, 233, 97, 107, 47, 64, 439, 63, 238, 47, 212, 129, 219, 45, 63, 88, 107, 14]
[403, 30, 84, 391, 61, 104, 400, 32, 291, 84, 440, 121, 243]
[46, 68, 67, 39, 73, 129, 73, 45, 303, 180]
[42, 103, 103, 296, 207, 87, 130]
[69, 78, 58, 86]
[40, 386, 440, 253]
[41, 371, 324, 271]
[40, 75, 248, 31, 65, 71, 389, 327]
[44, 253, 91, 57, 133, 39, 60, 

In [22]:
words = answer_tokenizer.detokenize(token_batch)
print('Проверим обратное преобразование:')

# Объединение полученного тензора в текст, объединяем пробелами
print(tf.strings.reduce_join(words, separator=' ', axis=-1))

Проверим обратное преобразование:
tf.Tensor(
[b'joe flacco' b'whig party' b'utc + 07 : 00' ... b'qingtian dialect'
 b'italy' b'overbrook high school'], shape=(3778,), dtype=string)


In [24]:
lengths = []

for pt_examples, en_examples in train_examples.batch(1024):
  pt_tokens = question_tokenizer.tokenize(pt_examples)
  lengths.append(pt_tokens.row_lengths())

  en_tokens = answer_tokenizer.tokenize(en_examples)
  lengths.append(en_tokens.row_lengths())

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot batch tensors with different shapes in component 0. First element had shape [1] and element 3 had shape [10]. [Op:IteratorGetNext] name: 